In [ ]:
!unzip /content/Face-Images

In [66]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
import torch.nn as nn

In [65]:
trans = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor()
])

In [67]:
TRAIN_DATA_PATH = "/content/Face_Images/Final_Training_Images"
TEST_DATA_PATH = "/content/Face_Images/Final_Training_Images"
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=trans)
train_data_loader = data.DataLoader(train_data, batch_size=4, shuffle=True)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=trans)
test_data_loader  = data.DataLoader(test_data, batch_size=4, shuffle=True)

In [68]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)
AlexNet_model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [69]:
AlexNet_model.classifier[4] = nn.Linear(4096,1024)
AlexNet_model.classifier[6] = nn.Linear(1024,16)

In [ ]:
#Instantiating CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Verifying CUDA
print(device)

#Move the input and AlexNet_model to GPU for speed if available
AlexNet_model.to(device)

In [71]:
import torch.optim as optim
import torch.nn as nn

#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.001, momentum=0.9)

In [75]:
for epoch in range(10):
  running_loss = 0.0
  for i, data in enumerate(train_data_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data[0].to(device), data[1].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    output = AlexNet_model(inputs)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 2000))

print('Finished Training of AlexNet')

[1,     1] loss: 0.000
[1,     2] loss: 0.000
[1,     3] loss: 0.000
[1,     4] loss: 0.000
[1,     5] loss: 0.000
[1,     6] loss: 0.000
[1,     7] loss: 0.000
[1,     8] loss: 0.000
[1,     9] loss: 0.000
[1,    10] loss: 0.000
[1,    11] loss: 0.000
[1,    12] loss: 0.000
[1,    13] loss: 0.000
[1,    14] loss: 0.000
[1,    15] loss: 0.000
[1,    16] loss: 0.000
[2,     1] loss: 0.000
[2,     2] loss: 0.000
[2,     3] loss: 0.000
[2,     4] loss: 0.000
[2,     5] loss: 0.000
[2,     6] loss: 0.000
[2,     7] loss: 0.000
[2,     8] loss: 0.000
[2,     9] loss: 0.000
[2,    10] loss: 0.000
[2,    11] loss: 0.000
[2,    12] loss: 0.000
[2,    13] loss: 0.000
[2,    14] loss: 0.000
[2,    15] loss: 0.000
[2,    16] loss: 0.000
[3,     1] loss: 0.000
[3,     2] loss: 0.000
[3,     3] loss: 0.000
[3,     4] loss: 0.000
[3,     5] loss: 0.000
[3,     6] loss: 0.000
[3,     7] loss: 0.000
[3,     8] loss: 0.000
[3,     9] loss: 0.000
[3,    10] loss: 0.000
[3,    11] loss: 0.000
[3,    12] 

In [76]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in test_data_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = AlexNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 100 %
